In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, f1_score
from pathlib import Path
import random

In [5]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

In [6]:
DATA_DIR = Path("/Users/alvarosanchez/Downloads/MURA-v1.1")
TRAIN_DIR = DATA_DIR / "train"
VALID_DIR = DATA_DIR / "valid"

In [7]:
# HIPERPARÁMETROS
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-4
PATIENCE = 4

In [8]:
# TRANSFORMACIONES
transform_train = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [9]:
#CARGA DE DATOS
train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=transform_train)
valid_dataset = datasets.ImageFolder(VALID_DIR, transform=transform_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE)

In [10]:
#MODELO
model = models.resnet18(weights="DEFAULT")
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(DEVICE)

In [11]:
#FUNCIÓN DE PÉRDIDA Y OPTIMIZADOR
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

In [15]:
#ENTRENAMIENTO CON EARLY STOPPING
train_losses, val_losses = [], []
best_val_loss = float('inf')
epochs_no_improve = 0

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    #VALIDACIÓN
    model.eval()
    val_loss = 0.0
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    avg_val_loss = val_loss / len(valid_loader)
    val_losses.append(avg_val_loss)
    scheduler.step(avg_val_loss)
    f1 = f1_score(y_true, y_pred,average='weighted')
    acc = np.mean(np.array(y_true) == np.array(y_pred))

    print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Acc: {acc:.4f} | F1: {f1:.4f}")

    #GUARDADO DEL MEJOR MODELO
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), "/Users/alvarosanchez/ONLINE_DS_THEBRIDGE_ALVAROSMMS-1/ML_Clasificacion_Radiografias_Muscoesqueleticas/src/models/resnet18_optimized.pt")
        print("Mejor modelo guardado.")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print("Early stopping activado.")
            break

Epoch 1 | Train Loss: 0.0036 | Val Loss: 0.0012 | Acc: 0.2893 | F1: 0.1304
Mejor modelo guardado.
Epoch 2 | Train Loss: 0.0033 | Val Loss: 0.0017 | Acc: 0.2890 | F1: 0.1305
Epoch 3 | Train Loss: 0.0016 | Val Loss: 0.0006 | Acc: 0.2893 | F1: 0.1304
Mejor modelo guardado.
Epoch 4 | Train Loss: 0.0011 | Val Loss: 0.0014 | Acc: 0.2890 | F1: 0.1305
Epoch 5 | Train Loss: 0.0016 | Val Loss: 0.0016 | Acc: 0.2893 | F1: 0.1304
Epoch 6 | Train Loss: 0.0011 | Val Loss: 0.0025 | Acc: 0.2887 | F1: 0.1298
Epoch 7 | Train Loss: 0.0004 | Val Loss: 0.0021 | Acc: 0.2893 | F1: 0.1300
Early stopping activado.


In [16]:
#CLASIFICACIÓN FINAL
print("\nReporte de clasificación final:")
print(classification_report(y_true, y_pred, target_names=train_dataset.classes))


Reporte de clasificación final:
              precision    recall  f1-score   support

    XR_ELBOW       0.29      1.00      0.45       465
   XR_FINGER       0.29      1.00      0.45       461
  XR_FOREARM       0.00      0.00      0.00       301
     XR_HAND       0.00      0.00      0.00       460
  XR_HUMERUS       0.00      0.00      0.00       288
 XR_SHOULDER       0.00      0.00      0.00       563
    XR_WRIST       0.00      0.00      0.00       659

    accuracy                           0.29      3197
   macro avg       0.08      0.29      0.13      3197
weighted avg       0.08      0.29      0.13      3197



/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import matplotlib.pyplot as plt

#FUNCIÓN PARA MOSTRAR PREDICCIONES
def mostrar_predicciones(modelo, dataloader, class_names, n=8):
    modelo.eval()
    images_shown = 0
    plt.figure(figsize=(15, 8))
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = modelo(inputs)
            preds = torch.argmax(outputs, dim=1)

            for i in range(inputs.size(0)):
                if images_shown == n:
                    break

                img = inputs[i].cpu().permute(1, 2, 0).numpy()
                img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                img = np.clip(img, 0, 1)

                true_label = class_names[labels[i]]
                pred_label = class_names[preds[i]]

                color = "green" if true_label == pred_label else "red"
                title = f"Pred: {pred_label}\nGT: {true_label}"

                plt.subplot(n // 4 + 1, 4, images_shown + 1)
                plt.imshow(img)
                plt.title(title, color=color)
                plt.axis('off')
                
                images_shown += 1

            if images_shown == n:
                break
    
    plt.suptitle("Predicciones del modelo en validación", fontsize=16)
    plt.tight_layout()
    plt.show()

#LLAMADA A LA FUNCIÓN
mostrar_predicciones(model, valid_loader, train_dataset.classes, n=8)